# CHESS DATA ANALYSIS

TEAM: Abraham Borg, Mehar Rekhi, Sarom Thin, Cristian Vazquez

INTRO: We want to analyze a data set of chess games and use the information to make predictions about who will win a game of chess. The data is taken from this source: https://www.kaggle.com/datasets/milesh1/35-million-chess-games?resource=download 

#### DESCRIPTION OF SOURCE DATA SET 
The data shows all game moves for White and Black. The date of each game is specified. We are also given the results of each game. We know how many moves each player made. Finally, we know what the ELO rating of each player was at the time of the game. There is a lot of information that we are not interested in so that will need to be cleaned up. Also, the list of moves is a long string, but we want to have each move occupy its own dataframe column.

The complete dataset is 3.5 million games (filesize about 2GB), but for this experiment we initially selected the first 30,000 games for analysis. It was clear that this sample was heavily skewed for White. So instead we opted to randomly select games.

The data set was loaded into the dataframe from the local machine, and then the pickle of the completely prepared dataset was loaded onto github.

#### PREDICTION AND FEATURES TO USE AS PREDICTORS
PREDICTORS: The openings for White (first 8 moves), W-ELO, B-ELO, and Num Moves

PREDICT: Game Result, will White win based on the the predictors in our model?

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from matplotlib import rcParams
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html

sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7

# DATA PREPARATION

The data set is given with many columns that we don't need, data in the wrong format, and many records that are not useful. The data file is also very big. To make things easier we upload the data from the local machine and then generate a PKL file of the prepared dataframe. The full data set is hosted here, https://archive.org/download/chess-data, and the csv file is here, https://ia902501.us.archive.org/23/items/chess-data/Chess_Data.txt

In [4]:
filename = 'C:\\Users\\Abrah\\Desktop\\CST383 - Data Science\\Final Project\\Chess_Data.txt'
n = 3561470     #number of records in file
s = 450000      #desired sample size
skip = sorted(random.sample(range(n), n - s))

In [5]:
# column names, without the chess moves column
misc_columnNames = ['PNG_File_Pos - DELETE ME', 'Date', 'Game Result', 'W-ELO', 'B-ELO', 
                    'Num Moves', 'miscDate - DELETE ME', 'result - DELETE ME', 'wELO - DELETE ME', 'bELO - DELETE ME', 
                    'event date - DELETE ME', 'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 
                    'bad len - DELETE ME']
delete_cols = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME']

In [6]:
# read all data except chess moves,
misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)
misc_chess_data.drop(misc_chess_data.columns[16], axis = 1, inplace = True)
misc_chess_data.columns = misc_columnNames
misc_chess_data.drop(labels = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME'], axis = 1, inplace = True)


C:\Users\Abrah\AppData\Local\Temp\ipykernel_20288\3410824919.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)


In [7]:
misc_chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves
0,2000.03.14,1-0,2851,None,53
1,2000.06.19,1/2-1/2,2851,None,46
2,2000.06.19,1-0,2851,None,43
3,2000.03.04,1/2-1/2,2851,2725,76
4,2000.03.14,1-0,2851,None,47


In the above cell we can see that the B-ELO has None values. We will fix that shortly. Next we prepare isolate the chess moves from the other data to make it easier to prepare this portion of the data. The dataframe game_moves will be merged with the previous dataframe after processsing.

In [8]:
# Isolate game moves from everything else.
game_moves = pd.read_csv(filename, engine = 'python', sep = '###', on_bad_lines = 'skip', header = None, skiprows = skip)

# drop first column of game moves (this is the misc chess data)
game_moves.drop(game_moves.columns[0], axis = 1, inplace = True)

In [9]:
game_moves.head()

,1
0,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4....
1,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bc4 B3.Bc5 W4.c3...
2,W1.e4 B1.c5 W2.Nf3 B2.d6 W3.c3 B3.Nf6 W4.Be2 ...
3,W1.d4 B1.Nf6 W2.Nf3 B2.g6 W3.c4 B3.Bg7 W4.Nc3...
4,W1.d4 B1.Nf6 W2.Bg5 B2.g6 W3.Bxf6 B3.exf6 W4....


In [10]:
# split game moves df into columns for each move. 
game_moves = game_moves.iloc[:, 0].str.lstrip()
game_moves = game_moves.iloc[:].str.split(pat = ' ', expand = True)
game_moves.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557
0,W1.e4,B1.d5,W2.exd5,B2.Qxd5,W3.Nc3,B3.Qa5,W4.d4,B4.Nf6,W5.Nf3,B5.c6,W6.Ne5,B6.Bf5,W7.g4,B7.Be4,W8.f3,B8.Bd5,W9.a3,B9.Nbd7,W10.Be3,B10.Nxe5,W11.dxe5,B11.Nxg4,W12.Bd4,B12.e6,W13.b4,B13.Qd8,W14.Nxd5,B14.Qxd5,W15.c4,B15.Ne3,W16.cxd5,B16.Nxd1,W17.dxc6,B17.bxc6,W18.Rxd1,B18.Be7,W19.Ba6,B19.O-O,W20.Ke2,B20.Rab8,W21.Rc1,B21.Rfd8,W22.Rhd1,B22.c5,W23.Bxc5,B23.Rxd1,W24.Rxd1,B24.Bxc5,W25.bxc5,B25.g6,W26.c6,B26.Rb2+,W27.Rd2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None

In [11]:
# chess moves need to be strings, so convert objects to strings.
game_moves = game_moves.astype('string', copy=True, errors='raise')
game_moves.dtypes

0      string
1      string
2      string
3      string
4      string
        ...  
553    string
554    string
555    string
556    string
557    string
Length: 558, dtype: object

In [12]:
# merge misc data and game moves into one df.
chess_data = pd.concat([misc_chess_data, game_moves], axis = 1)

In [13]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557
0,2000.03.14,1-0,2851,None,53,W1.e4,B1.d5,W2.exd5,B2.Qxd5,W3.Nc3,B3.Qa5,W4.d4,B4.Nf6,W5.Nf3,B5.c6,W6.Ne5,B6.Bf5,W7.g4,B7.Be4,W8.f3,B8.Bd5,W9.a3,B9.Nbd7,W10.Be3,B10.Nxe5,W11.dxe5,B11.Nxg4,W12.Bd4,B12.e6,W13.b4,B13.Qd8,W14.Nxd5,B14.Qxd5,W15.c4,B15.Ne3,W16.cxd5,B16.Nxd1,W17.dxc6,B17.bxc6,W18.Rxd1,B18.Be7,W19.Ba6,B19.O-O,W20.Ke2,B20.Rab8,W21.Rc1,B21.Rfd8,W22.Rhd1,B22.c5,W23.Bxc5,B23.Rxd1,W24.Rxd1,B24.Bxc5,W25.bxc5,B25.g6,W26.c6,B26.Rb2+,W27.Rd2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

The merging of the dataframes was successful. There are a few remaining steps for data preparation. We need to make sure we have the correct datatypes in place and also remove rows with missing data or unnecessary data.

In [14]:
# remove rows with missing player ratings.
chess_data = chess_data[chess_data['B-ELO'] != 'None']
chess_data = chess_data[chess_data['W-ELO'] != 'None']

In [15]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557
3,2000.03.04,1/2-1/2,2851,2725,76,W1.d4,B1.Nf6,W2.Nf3,B2.g6,W3.c4,B3.Bg7,W4.Nc3,B4.d5,W5.Qb3,B5.dxc4,W6.Qxc4,B6.O-O,W7.e4,B7.a6,W8.Be2,B8.b5,W9.Qb3,B9.c5,W10.dxc5,B10.Bb7,W11.O-O,B11.Nxe4,W12.Nxe4,B12.Bxe4,W13.Bg5,B13.Nc6,W14.Qe3,B14.Qd5,W15.Rad1,B15.Qe6,W16.Bh6,B16.Bf5,W17.Bxg7,B17.Qxe3,W18.fxe3,B18.Kxg7,W19.a3,B19.Rfd8,W20.b4,B20.Bc2,W21.Rxd8,B21.Rxd8,W22.Rc1,B22.Be4,W23.Kf2,B23.Kf6,W24.g4,B24.h6,W25.h4,B25.Ke6,W26.Rd1,B26.Bxf3,W27.Rxd8,B27.Bxe2,W28.Rc8,B28.Kd7,W29.Rxc6,B29.Kxc6,W30.Kxe2,B30.f6,W31.Kd3,B31.e6,W32.Kd4,B32.g5,W33.h5,B33.Kd7,W34.e4,B34.Kc6,W35.e5,B35.f5,W36.gxf5,B36.exf5,W37.e6,B37.f4,W38.Ke4,B38.Kc7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

In [16]:
# We are interested in games that show wins or losses. We are not interested in draws. So we take out drawn games.
chess_data = chess_data[chess_data['Game Result'] != '1/2-1/2']

In [17]:
# remove rows where the number of game moves is 0.
chess_data = chess_data[chess_data['Num Moves'] != 0]

In [18]:
# remove rows where the number of game moves is over 75 for each player (75 * 2 players)
chess_data = chess_data[chess_data['Num Moves'] <= 150]

In [19]:
# we only want games with openings that we can analyze, so get all games that 
# have num moves at least 16 (8 move each side)
chess_data = chess_data[chess_data['Num Moves'] >= 16]

In [20]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557
7,2000.05.27,1-0,2850,2675,67,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.Nf6,W4.Nc3,B4.cxd4,W5.Nxd4,B5.a6,W6.Be3,B6.Nbd7,W7.f3,B7.g6,W8.Qd2,B8.b5,W9.a4,B9.bxa4,W10.Nxa4,B10.Bg7,W11.Be2,B11.Bb7,W12.c4,B12.O-O,W13.O-O,B13.Nc5,W14.b4,B14.e5,W15.bxc5,B15.exd4,W16.Bg5,B16.Re8,W17.cxd6,B17.Qxd6,W18.c5,B18.Qc6,W19.Bc4,B19.Bc8,W20.Nb6,B20.Ra7,W21.Rfc1,B21.Nd7,W22.Bd5,B22.Qc7,W23.Nc4,B23.Qxc5,W24.Na5,B24.Qf8,W25.Nc6,B25.h6,W26.Bh4,B26.Rc7,W27.Be7,B27.Rxc6,W28.Bxf8,B28.Rxc1+,W29.Rxc1,B29.Nxf8,W30.Qf4,B30.Be6,W31.Rc7,B31.Bxd5,W32.exd5,B32.f5,W33.d6,B33.Rd8,W34.Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<

In [21]:
# we have too many columns because of all the NA values. Get dataframe up to col 155. That's 5 cols for misc data plus 150 cols
# for the chess moves (there will be a max of 150 moves)
chess_data = chess_data[chess_data.columns[:155]]

In [22]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
7,2000.05.27,1-0,2850,2675,67,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.Nf6,W4.Nc3,B4.cxd4,W5.Nxd4,B5.a6,W6.Be3,B6.Nbd7,W7.f3,B7.g6,W8.Qd2,B8.b5,W9.a4,B9.bxa4,W10.Nxa4,B10.Bg7,W11.Be2,B11.Bb7,W12.c4,B12.O-O,W13.O-O,B13.Nc5,W14.b4,B14.e5,W15.bxc5,B15.exd4,W16.Bg5,B16.Re8,W17.cxd6,B17.Qxd6,W18.c5,B18.Qc6,W19.Bc4,B19.Bc8,W20.Nb6,B20.Ra7,W21.Rfc1,B21.Nd7,W22.Bd5,B22.Qc7,W23.Nc4,B23.Qxc5,W24.Na5,B24.Qf8,W25.Nc6,B25.h6,W26.Bh4,B26.Rc7,W27.Be7,B27.Rxc6,W28.Bxf8,B28.Rxc1+,W29.Rxc1,B29.Nxf8,W30.Qf4,B30.Be6,W31.Rc7,B31.Bxd5,W32.exd5,B32.f5,W33.d6,B33.Rd8,W34.Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,2001.01.26,1-0,2849,2629,93,W1.c4,B1.e5,W2.Nc3,B2.Nf6,W3.Nf3,B3.Nc6,W4.g3,B4.d5,W5.cxd5,B5.Nxd5,W6.Bg2,B6.Nb6,W7.O-O,B7.Be7,W8.Rb1,B8.O-O,W9.b4,B9.Nxb4,W10.Nxe5,B10.c6,W11.a3,B11.N4d5,W12.Nxd5,B12.cxd5,W13.a4,B13.Bf6,W14.d4,B14.Bf5,W15.Rb5,B15.a6,W16.Rb2,B16.Rc8,W17.Bf4,B17.Bxe5,W18.Bxe5,B18.Re8,W19.e3,B19.Re7,W20.g4,B20.Be4,W21.f3,B21.Nc4,W22.Re2,B22.Bg6,W23.h4,B23.f6,W24.Bf4,B24.Qd7,W25.Rfe1,B25.Rc6,W26.h5,B26.Bf7,W27.e4,B27.dxe4,W28.Rxe4,B28.Rxe4,W29.Rxe4,B29.Bd5,W30.Re2,B30.Rc8,W31.h6,B31.g5,W32.Bg3,B32.Re8,W33.Qe1,B33.Rxe2,W34.Qxe2,B34.Kf8,W35.Qd3,B35.Nb6,W36.a5,B36.Nc4,W37.Bf1,B37.Qe7,W38.Qc3,B38.Qe3+,W39.Qxe3,B39.Nxe3,W40.Bd3,B40.Nc4,W41.Be1,B41.Kg8,W42.Kf2,B42.b5,W43.Bb4,B43.Nb2,W44.Bf5,B44.Bc4,W45.Ke3,B45.Bd5,W46.Bc8,B46.Nc4+,W47.Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19,2003.03.01,1-0,2847,2753,123,W1.e4,B1.c6,W2.d4,B2.d5,W3.Nd2,B3.dxe4,W4.Nxe4,B4.Bf5,W5.Ng3,B5.Bg6,W6.h4,B6.h6,W7.Nf3,B7.Nd7,W8.h5,B8.Bh7,W9.Bd3,B9.Bxd3,W10.Qxd3,B10.Ngf6,W11.Bf4,B11.e6,W12.O-O-O,B12.Be7,W13.Kb1,B13.O-O,W14.Ne4,B14.Nxe4,W15.Qxe4,B15.Nf6,W16.Qe2,B16.Qd5,W17.Ne5,B17.Qe4,W18.Qxe4,B18.Nxe4,W19.Rhe1,B19.Nf6,W20.g4,B20.Rfd8,W21.Be3,B21.Bd6,W22.f3,B22.Rac8,W23.c4,B23.a5,W24.a4,B24.Nd7,W25.Bd2,B25.Bc7,W26.Bc3,B26.Nxe5,W27.dxe5,B27.c5,W28.Kc2,B28.Rxd1,W29.Kxd1,B29.Rd8+,W30.Ke2,B30.Rd7,W31.f4,B31.Bd8,W32.f5,B32.Bg5,W33.f6,B33.b6,W34.Ra1,B34.Rd8,W35.Ra3,B35.gxf6,W36.Rb3,B36.Bf4,W37.Rxb6,B37.Bxe5,W38.Bxe5,B38.fxe5,W39.Rb5,B39.Rd4,W40.Rxc5,B40.Rxg4,W41.b3,B41.Rg3,W42.Rxe5,B42.Rxb3,W43.Rxa5,B43.f5,W44.Ra8+,B44.Kg7,W45.c5,B45.Rc3,W46.Rc8,B46.Ra3,W47.c6,B47.Rxa4,W48.Re8,B48.Rc4,W49.Rxe6,B49.f4,W50.Rg6+,B50.Kh7,W51.Kd3,B51.Rc5,W52.Kd4,B52.Rc1,W53.Ke4,B53.Rc4+,W54.Kd5,B54.Rc3,W55.Kd4,B55.Rc1,W56.Ke4,B56.Rc4+,W57.Kf3,B57.Kh8,W58.Rxh6+,B58.Kg7,W59.Rd6,B59.Kh7,W60.Kg4,B60.Kg7,W61.Rd7+,B61.Kf6,W62.c7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21,2001.10.18,1-0,2838,2631,85,W1.e4,B1.c5,W2.Nf3,B2.e6,W3.d4,B3.cxd4,W4.Nxd4,B4.a6,W5.Nc3,B5.Qc7,W6.Bd3,B6.Nc6,W7.Nxc6,B7.bxc6,W8.O-O,B8.Nf6,W9.Qe2,B9.Bd6,W10.f4,B10.e5,W11.Kh1,B11.h6,W12.Bd2,B12.O-O,W13.f5,B13.Re8,W14.Bc4,B14.a5,W15.Rf3,B15.Bb4,W16.Rg3,B16.Kf8,W17.Qe3,B17.Ng8,W18.f6,B18.gxf6,W19.Rf1,B19.Bxc3,W20.

In [23]:
# we only care about the year that the game took place. reformat the date col to reflect that
# try a string function on the df.
new_column = chess_data['Date'].str.slice(0, 4, 1)
chess_data['Date'] = new_column

In [24]:
chess_data.shape

(156645, 155)

In [25]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
7,2000,1-0,2850,2675,67,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.Nf6,W4.Nc3,B4.cxd4,W5.Nxd4,B5.a6,W6.Be3,B6.Nbd7,W7.f3,B7.g6,W8.Qd2,B8.b5,W9.a4,B9.bxa4,W10.Nxa4,B10.Bg7,W11.Be2,B11.Bb7,W12.c4,B12.O-O,W13.O-O,B13.Nc5,W14.b4,B14.e5,W15.bxc5,B15.exd4,W16.Bg5,B16.Re8,W17.cxd6,B17.Qxd6,W18.c5,B18.Qc6,W19.Bc4,B19.Bc8,W20.Nb6,B20.Ra7,W21.Rfc1,B21.Nd7,W22.Bd5,B22.Qc7,W23.Nc4,B23.Qxc5,W24.Na5,B24.Qf8,W25.Nc6,B25.h6,W26.Bh4,B26.Rc7,W27.Be7,B27.Rxc6,W28.Bxf8,B28.Rxc1+,W29.Rxc1,B29.Nxf8,W30.Qf4,B30.Be6,W31.Rc7,B31.Bxd5,W32.exd5,B32.f5,W33.d6,B33.Rd8,W34.Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,2001,1-0,2849,2629,93,W1.c4,B1.e5,W2.Nc3,B2.Nf6,W3.Nf3,B3.Nc6,W4.g3,B4.d5,W5.cxd5,B5.Nxd5,W6.Bg2,B6.Nb6,W7.O-O,B7.Be7,W8.Rb1,B8.O-O,W9.b4,B9.Nxb4,W10.Nxe5,B10.c6,W11.a3,B11.N4d5,W12.Nxd5,B12.cxd5,W13.a4,B13.Bf6,W14.d4,B14.Bf5,W15.Rb5,B15.a6,W16.Rb2,B16.Rc8,W17.Bf4,B17.Bxe5,W18.Bxe5,B18.Re8,W19.e3,B19.Re7,W20.g4,B20.Be4,W21.f3,B21.Nc4,W22.Re2,B22.Bg6,W23.h4,B23.f6,W24.Bf4,B24.Qd7,W25.Rfe1,B25.Rc6,W26.h5,B26.Bf7,W27.e4,B27.dxe4,W28.Rxe4,B28.Rxe4,W29.Rxe4,B29.Bd5,W30.Re2,B30.Rc8,W31.h6,B31.g5,W32.Bg3,B32.Re8,W33.Qe1,B33.Rxe2,W34.Qxe2,B34.Kf8,W35.Qd3,B35.Nb6,W36.a5,B36.Nc4,W37.Bf1,B37.Qe7,W38.Qc3,B38.Qe3+,W39.Qxe3,B39.Nxe3,W40.Bd3,B40.Nc4,W41.Be1,B41.Kg8,W42.Kf2,B42.b5,W43.Bb4,B43.Nb2,W44.Bf5,B44.Bc4,W45.Ke3,B45.Bd5,W46.Bc8,B46.Nc4+,W47.Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19,2003,1-0,2847,2753,123,W1.e4,B1.c6,W2.d4,B2.d5,W3.Nd2,B3.dxe4,W4.Nxe4,B4.Bf5,W5.Ng3,B5.Bg6,W6.h4,B6.h6,W7.Nf3,B7.Nd7,W8.h5,B8.Bh7,W9.Bd3,B9.Bxd3,W10.Qxd3,B10.Ngf6,W11.Bf4,B11.e6,W12.O-O-O,B12.Be7,W13.Kb1,B13.O-O,W14.Ne4,B14.Nxe4,W15.Qxe4,B15.Nf6,W16.Qe2,B16.Qd5,W17.Ne5,B17.Qe4,W18.Qxe4,B18.Nxe4,W19.Rhe1,B19.Nf6,W20.g4,B20.Rfd8,W21.Be3,B21.Bd6,W22.f3,B22.Rac8,W23.c4,B23.a5,W24.a4,B24.Nd7,W25.Bd2,B25.Bc7,W26.Bc3,B26.Nxe5,W27.dxe5,B27.c5,W28.Kc2,B28.Rxd1,W29.Kxd1,B29.Rd8+,W30.Ke2,B30.Rd7,W31.f4,B31.Bd8,W32.f5,B32.Bg5,W33.f6,B33.b6,W34.Ra1,B34.Rd8,W35.Ra3,B35.gxf6,W36.Rb3,B36.Bf4,W37.Rxb6,B37.Bxe5,W38.Bxe5,B38.fxe5,W39.Rb5,B39.Rd4,W40.Rxc5,B40.Rxg4,W41.b3,B41.Rg3,W42.Rxe5,B42.Rxb3,W43.Rxa5,B43.f5,W44.Ra8+,B44.Kg7,W45.c5,B45.Rc3,W46.Rc8,B46.Ra3,W47.c6,B47.Rxa4,W48.Re8,B48.Rc4,W49.Rxe6,B49.f4,W50.Rg6+,B50.Kh7,W51.Kd3,B51.Rc5,W52.Kd4,B52.Rc1,W53.Ke4,B53.Rc4+,W54.Kd5,B54.Rc3,W55.Kd4,B55.Rc1,W56.Ke4,B56.Rc4+,W57.Kf3,B57.Kh8,W58.Rxh6+,B58.Kg7,W59.Rd6,B59.Kh7,W60.Kg4,B60.Kg7,W61.Rd7+,B61.Kf6,W62.c7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21,2001,1-0,2838,2631,85,W1.e4,B1.c5,W2.Nf3,B2.e6,W3.d4,B3.cxd4,W4.Nxd4,B4.a6,W5.Nc3,B5.Qc7,W6.Bd3,B6.Nc6,W7.Nxc6,B7.bxc6,W8.O-O,B8.Nf6,W9.Qe2,B9.Bd6,W10.f4,B10.e5,W11.Kh1,B11.h6,W12.Bd2,B12.O-O,W13.f5,B13.Re8,W14.Bc4,B14.a5,W15.Rf3,B15.Bb4,W16.Rg3,B16.Kf8,W17.Qe3,B17.Ng8,W18.f6,B18.gxf6,W19.Rf1,B19.Bxc3,W20.Rxg8+,B20.Kxg8,W21.Qg3+,

In [26]:
# take out games without dates.
chess_data = chess_data[chess_data['Date'].str.isnumeric()]

In [27]:
# take out any that show * for Game Result
chess_data = chess_data[chess_data['Game Result'] != '*']

In [28]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
7,2000,1-0,2850,2675,67,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.Nf6,W4.Nc3,B4.cxd4,W5.Nxd4,B5.a6,W6.Be3,B6.Nbd7,W7.f3,B7.g6,W8.Qd2,B8.b5,W9.a4,B9.bxa4,W10.Nxa4,B10.Bg7,W11.Be2,B11.Bb7,W12.c4,B12.O-O,W13.O-O,B13.Nc5,W14.b4,B14.e5,W15.bxc5,B15.exd4,W16.Bg5,B16.Re8,W17.cxd6,B17.Qxd6,W18.c5,B18.Qc6,W19.Bc4,B19.Bc8,W20.Nb6,B20.Ra7,W21.Rfc1,B21.Nd7,W22.Bd5,B22.Qc7,W23.Nc4,B23.Qxc5,W24.Na5,B24.Qf8,W25.Nc6,B25.h6,W26.Bh4,B26.Rc7,W27.Be7,B27.Rxc6,W28.Bxf8,B28.Rxc1+,W29.Rxc1,B29.Nxf8,W30.Qf4,B30.Be6,W31.Rc7,B31.Bxd5,W32.exd5,B32.f5,W33.d6,B33.Rd8,W34.Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,2001,1-0,2849,2629,93,W1.c4,B1.e5,W2.Nc3,B2.Nf6,W3.Nf3,B3.Nc6,W4.g3,B4.d5,W5.cxd5,B5.Nxd5,W6.Bg2,B6.Nb6,W7.O-O,B7.Be7,W8.Rb1,B8.O-O,W9.b4,B9.Nxb4,W10.Nxe5,B10.c6,W11.a3,B11.N4d5,W12.Nxd5,B12.cxd5,W13.a4,B13.Bf6,W14.d4,B14.Bf5,W15.Rb5,B15.a6,W16.Rb2,B16.Rc8,W17.Bf4,B17.Bxe5,W18.Bxe5,B18.Re8,W19.e3,B19.Re7,W20.g4,B20.Be4,W21.f3,B21.Nc4,W22.Re2,B22.Bg6,W23.h4,B23.f6,W24.Bf4,B24.Qd7,W25.Rfe1,B25.Rc6,W26.h5,B26.Bf7,W27.e4,B27.dxe4,W28.Rxe4,B28.Rxe4,W29.Rxe4,B29.Bd5,W30.Re2,B30.Rc8,W31.h6,B31.g5,W32.Bg3,B32.Re8,W33.Qe1,B33.Rxe2,W34.Qxe2,B34.Kf8,W35.Qd3,B35.Nb6,W36.a5,B36.Nc4,W37.Bf1,B37.Qe7,W38.Qc3,B38.Qe3+,W39.Qxe3,B39.Nxe3,W40.Bd3,B40.Nc4,W41.Be1,B41.Kg8,W42.Kf2,B42.b5,W43.Bb4,B43.Nb2,W44.Bf5,B44.Bc4,W45.Ke3,B45.Bd5,W46.Bc8,B46.Nc4+,W47.Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19,2003,1-0,2847,2753,123,W1.e4,B1.c6,W2.d4,B2.d5,W3.Nd2,B3.dxe4,W4.Nxe4,B4.Bf5,W5.Ng3,B5.Bg6,W6.h4,B6.h6,W7.Nf3,B7.Nd7,W8.h5,B8.Bh7,W9.Bd3,B9.Bxd3,W10.Qxd3,B10.Ngf6,W11.Bf4,B11.e6,W12.O-O-O,B12.Be7,W13.Kb1,B13.O-O,W14.Ne4,B14.Nxe4,W15.Qxe4,B15.Nf6,W16.Qe2,B16.Qd5,W17.Ne5,B17.Qe4,W18.Qxe4,B18.Nxe4,W19.Rhe1,B19.Nf6,W20.g4,B20.Rfd8,W21.Be3,B21.Bd6,W22.f3,B22.Rac8,W23.c4,B23.a5,W24.a4,B24.Nd7,W25.Bd2,B25.Bc7,W26.Bc3,B26.Nxe5,W27.dxe5,B27.c5,W28.Kc2,B28.Rxd1,W29.Kxd1,B29.Rd8+,W30.Ke2,B30.Rd7,W31.f4,B31.Bd8,W32.f5,B32.Bg5,W33.f6,B33.b6,W34.Ra1,B34.Rd8,W35.Ra3,B35.gxf6,W36.Rb3,B36.Bf4,W37.Rxb6,B37.Bxe5,W38.Bxe5,B38.fxe5,W39.Rb5,B39.Rd4,W40.Rxc5,B40.Rxg4,W41.b3,B41.Rg3,W42.Rxe5,B42.Rxb3,W43.Rxa5,B43.f5,W44.Ra8+,B44.Kg7,W45.c5,B45.Rc3,W46.Rc8,B46.Ra3,W47.c6,B47.Rxa4,W48.Re8,B48.Rc4,W49.Rxe6,B49.f4,W50.Rg6+,B50.Kh7,W51.Kd3,B51.Rc5,W52.Kd4,B52.Rc1,W53.Ke4,B53.Rc4+,W54.Kd5,B54.Rc3,W55.Kd4,B55.Rc1,W56.Ke4,B56.Rc4+,W57.Kf3,B57.Kh8,W58.Rxh6+,B58.Kg7,W59.Rd6,B59.Kh7,W60.Kg4,B60.Kg7,W61.Rd7+,B61.Kf6,W62.c7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21,2001,1-0,2838,2631,85,W1.e4,B1.c5,W2.Nf3,B2.e6,W3.d4,B3.cxd4,W4.Nxd4,B4.a6,W5.Nc3,B5.Qc7,W6.Bd3,B6.Nc6,W7.Nxc6,B7.bxc6,W8.O-O,B8.Nf6,W9.Qe2,B9.Bd6,W10.f4,B10.e5,W11.Kh1,B11.h6,W12.Bd2,B12.O-O,W13.f5,B13.Re8,W14.Bc4,B14.a5,W15.Rf3,B15.Bb4,W16.Rg3,B16.Kf8,W17.Qe3,B17.Ng8,W18.f6,B18.gxf6,W19.Rf1,B19.Bxc3,W20.Rxg8+,B20.Kxg8,W21.Qg3+,

In [29]:
# we want the date to be a number.
chess_data = chess_data.astype({'Date': 'int'}, copy=True, errors='raise')

In [30]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
7,2000,1-0,2850,2675,67,W1.e4,B1.c5,W2.Nf3,B2.d6,W3.d4,B3.Nf6,W4.Nc3,B4.cxd4,W5.Nxd4,B5.a6,W6.Be3,B6.Nbd7,W7.f3,B7.g6,W8.Qd2,B8.b5,W9.a4,B9.bxa4,W10.Nxa4,B10.Bg7,W11.Be2,B11.Bb7,W12.c4,B12.O-O,W13.O-O,B13.Nc5,W14.b4,B14.e5,W15.bxc5,B15.exd4,W16.Bg5,B16.Re8,W17.cxd6,B17.Qxd6,W18.c5,B18.Qc6,W19.Bc4,B19.Bc8,W20.Nb6,B20.Ra7,W21.Rfc1,B21.Nd7,W22.Bd5,B22.Qc7,W23.Nc4,B23.Qxc5,W24.Na5,B24.Qf8,W25.Nc6,B25.h6,W26.Bh4,B26.Rc7,W27.Be7,B27.Rxc6,W28.Bxf8,B28.Rxc1+,W29.Rxc1,B29.Nxf8,W30.Qf4,B30.Be6,W31.Rc7,B31.Bxd5,W32.exd5,B32.f5,W33.d6,B33.Rd8,W34.Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,2001,1-0,2849,2629,93,W1.c4,B1.e5,W2.Nc3,B2.Nf6,W3.Nf3,B3.Nc6,W4.g3,B4.d5,W5.cxd5,B5.Nxd5,W6.Bg2,B6.Nb6,W7.O-O,B7.Be7,W8.Rb1,B8.O-O,W9.b4,B9.Nxb4,W10.Nxe5,B10.c6,W11.a3,B11.N4d5,W12.Nxd5,B12.cxd5,W13.a4,B13.Bf6,W14.d4,B14.Bf5,W15.Rb5,B15.a6,W16.Rb2,B16.Rc8,W17.Bf4,B17.Bxe5,W18.Bxe5,B18.Re8,W19.e3,B19.Re7,W20.g4,B20.Be4,W21.f3,B21.Nc4,W22.Re2,B22.Bg6,W23.h4,B23.f6,W24.Bf4,B24.Qd7,W25.Rfe1,B25.Rc6,W26.h5,B26.Bf7,W27.e4,B27.dxe4,W28.Rxe4,B28.Rxe4,W29.Rxe4,B29.Bd5,W30.Re2,B30.Rc8,W31.h6,B31.g5,W32.Bg3,B32.Re8,W33.Qe1,B33.Rxe2,W34.Qxe2,B34.Kf8,W35.Qd3,B35.Nb6,W36.a5,B36.Nc4,W37.Bf1,B37.Qe7,W38.Qc3,B38.Qe3+,W39.Qxe3,B39.Nxe3,W40.Bd3,B40.Nc4,W41.Be1,B41.Kg8,W42.Kf2,B42.b5,W43.Bb4,B43.Nb2,W44.Bf5,B44.Bc4,W45.Ke3,B45.Bd5,W46.Bc8,B46.Nc4+,W47.Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19,2003,1-0,2847,2753,123,W1.e4,B1.c6,W2.d4,B2.d5,W3.Nd2,B3.dxe4,W4.Nxe4,B4.Bf5,W5.Ng3,B5.Bg6,W6.h4,B6.h6,W7.Nf3,B7.Nd7,W8.h5,B8.Bh7,W9.Bd3,B9.Bxd3,W10.Qxd3,B10.Ngf6,W11.Bf4,B11.e6,W12.O-O-O,B12.Be7,W13.Kb1,B13.O-O,W14.Ne4,B14.Nxe4,W15.Qxe4,B15.Nf6,W16.Qe2,B16.Qd5,W17.Ne5,B17.Qe4,W18.Qxe4,B18.Nxe4,W19.Rhe1,B19.Nf6,W20.g4,B20.Rfd8,W21.Be3,B21.Bd6,W22.f3,B22.Rac8,W23.c4,B23.a5,W24.a4,B24.Nd7,W25.Bd2,B25.Bc7,W26.Bc3,B26.Nxe5,W27.dxe5,B27.c5,W28.Kc2,B28.Rxd1,W29.Kxd1,B29.Rd8+,W30.Ke2,B30.Rd7,W31.f4,B31.Bd8,W32.f5,B32.Bg5,W33.f6,B33.b6,W34.Ra1,B34.Rd8,W35.Ra3,B35.gxf6,W36.Rb3,B36.Bf4,W37.Rxb6,B37.Bxe5,W38.Bxe5,B38.fxe5,W39.Rb5,B39.Rd4,W40.Rxc5,B40.Rxg4,W41.b3,B41.Rg3,W42.Rxe5,B42.Rxb3,W43.Rxa5,B43.f5,W44.Ra8+,B44.Kg7,W45.c5,B45.Rc3,W46.Rc8,B46.Ra3,W47.c6,B47.Rxa4,W48.Re8,B48.Rc4,W49.Rxe6,B49.f4,W50.Rg6+,B50.Kh7,W51.Kd3,B51.Rc5,W52.Kd4,B52.Rc1,W53.Ke4,B53.Rc4+,W54.Kd5,B54.Rc3,W55.Kd4,B55.Rc1,W56.Ke4,B56.Rc4+,W57.Kf3,B57.Kh8,W58.Rxh6+,B58.Kg7,W59.Rd6,B59.Kh7,W60.Kg4,B60.Kg7,W61.Rd7+,B61.Kf6,W62.c7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21,2001,1-0,2838,2631,85,W1.e4,B1.c5,W2.Nf3,B2.e6,W3.d4,B3.cxd4,W4.Nxd4,B4.a6,W5.Nc3,B5.Qc7,W6.Bd3,B6.Nc6,W7.Nxc6,B7.bxc6,W8.O-O,B8.Nf6,W9.Qe2,B9.Bd6,W10.f4,B10.e5,W11.Kh1,B11.h6,W12.Bd2,B12.O-O,W13.f5,B13.Re8,W14.Bc4,B14.a5,W15.Rf3,B15.Bb4,W16.Rg3,B16.Kf8,W17.Qe3,B17.Ng8,W18.f6,B18.gxf6,W19.Rf1,B19.Bxc3,W20.Rxg8+,B20.Kxg8,W21.Qg3+,

In [31]:
chess_data.dtypes

Date            int32
Game Result    object
W-ELO          object
B-ELO          object
Num Moves       int64
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [32]:
# convert Num Moves to 32 bit int to reduce df size
chess_data = chess_data.astype({'Num Moves': 'int'}, copy=True, errors='raise')

In [33]:
# the dtypes for B-ELO and W-ELO need to be the same. So we convert W-ELO and B-ELO to int
chess_data = chess_data.astype({'B-ELO': 'int'}, copy=True, errors='raise')
chess_data = chess_data.astype({'W-ELO': 'int'}, copy=True, errors='raise')

In [34]:
# change game result to string
chess_data = chess_data.astype({'Game Result': 'string'}, copy=True, errors='raise')

In [35]:
chess_data.dtypes

Date            int32
Game Result    string
W-ELO           int32
B-ELO           int32
Num Moves       int32
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [36]:
# This code will be used to rename the col names.
def createMovesList1(x): # Number of moves is <= 9
  size = x.size
  return x.str.slice(3,size,1)
def createMovesList2(x): # Number of moves is <= 99
  size = x.size
  return x.str.slice(4,size,1)
def createMovesList3(x): # Number of moves is <= 100
  size = x.size
  return x.str.slice(5,size,1)

In [37]:
# using this code to rename the cols.
chess_data.iloc[:,5:23] = chess_data.iloc[:, 5:23].apply(createMovesList1)
chess_data.iloc[:,23:113] = chess_data.iloc[:, 23:113].apply(createMovesList2)
chess_data.iloc[:,113:] = chess_data.iloc[:, 113:].apply(createMovesList3)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
7,2000,1-0,2850,2675,67,e4,c5,Nf3,d6,d4,Nf6,Nc3,cxd4,Nxd4,a6,Be3,Nbd7,f3,g6,Qd2,b5,a4,bxa4,Nxa4,Bg7,Be2,Bb7,c4,O-O,O-O,Nc5,b4,e5,bxc5,exd4,Bg5,Re8,cxd6,Qxd6,c5,Qc6,Bc4,Bc8,Nb6,Ra7,Rfc1,Nd7,Bd5,Qc7,Nc4,Qxc5,Na5,Qf8,Nc6,h6,Bh4,Rc7,Be7,Rxc6,Bxf8,Rxc1+,Rxc1,Nxf8,Qf4,Be6,Rc7,Bxd5,exd5,f5,d6,Rd8,Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,2001,1-0,2849,2629,93,c4,e5,Nc3,Nf6,Nf3,Nc6,g3,d5,cxd5,Nxd5,Bg2,Nb6,O-O,Be7,Rb1,O-O,b4,Nxb4,Nxe5,c6,a3,N4d5,Nxd5,cxd5,a4,Bf6,d4,Bf5,Rb5,a6,Rb2,Rc8,Bf4,Bxe5,Bxe5,Re8,e3,Re7,g4,Be4,f3,Nc4,Re2,Bg6,h4,f6,Bf4,Qd7,Rfe1,Rc6,h5,Bf7,e4,dxe4,Rxe4,Rxe4,Rxe4,Bd5,Re2,Rc8,h6,g5,Bg3,Re8,Qe1,Rxe2,Qxe2,Kf8,Qd3,Nb6,a5,Nc4,Bf1,Qe7,Qc3,Qe3+,Qxe3,Nxe3,Bd3,Nc4,Be1,Kg8,Kf2,b5,Bb4,Nb2,Bf5,Bc4,Ke3,Bd5,Bc8,Nc4+,Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19,2003,1-0,2847,2753,123,e4,c6,d4,d5,Nd2,dxe4,Nxe4,Bf5,Ng3,Bg6,h4,h6,Nf3,Nd7,h5,Bh7,Bd3,Bxd3,Qxd3,Ngf6,Bf4,e6,O-O-O,Be7,Kb1,O-O,Ne4,Nxe4,Qxe4,Nf6,Qe2,Qd5,Ne5,Qe4,Qxe4,Nxe4,Rhe1,Nf6,g4,Rfd8,Be3,Bd6,f3,Rac8,c4,a5,a4,Nd7,Bd2,Bc7,Bc3,Nxe5,dxe5,c5,Kc2,Rxd1,Kxd1,Rd8+,Ke2,Rd7,f4,Bd8,f5,Bg5,f6,b6,Ra1,Rd8,Ra3,gxf6,Rb3,Bf4,Rxb6,Bxe5,Bxe5,fxe5,Rb5,Rd4,Rxc5,Rxg4,b3,Rg3,Rxe5,Rxb3,Rxa5,f5,Ra8+,Kg7,c5,Rc3,Rc8,Ra3,c6,Rxa4,Re8,Rc4,Rxe6,f4,Rg6+,Kh7,Kd3,Rc5,Kd4,Rc1,Ke4,Rc4+,Kd5,Rc3,d4,c1,e4,c4+,f3,h8,xh6+,g7,d6,h7,g4,g7,d7+,f6,7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21,2001,1-0,2838,2631,85,e4,c5,Nf3,e6,d4,cxd4,Nxd4,a6,Nc3,Qc7,Bd3,Nc6,Nxc6,bxc6,O-O,Nf6,Qe2,Bd6,f4,e5,Kh1,h6,Bd2,O-O,f5,Re8,Bc4,a5,Rf3,Bb4,Rg3,Kf8,Qe3,Ng8,f6,gxf6,Rf1,Bxc3,Rxg8+,Kxg8,Qg3+,Kf8,Rxf6,d5,Bxh6+,Ke7,Rxf7+,Kd6,Qg6+,Be6,Rxc7,Kxc7,exd5,cxd5,Bb5,Reb8,a4,Bd7,bxc3,Bxb5,axb5,Rxb5,h4,a4,Bg7,d4,Qf7+,Kb6,Qe6+,Kb7,Bxe5,Rxe5,Qxe5,a3,Qd5+,Kb8,cxd4,a2,Qb3+,Kc7,Qc3+,Kd7,Qa1,Kd6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
23,2001,1-0,2838,2802,99,d4,Nf6,c4,e6,Nc3,Bb4,Qc2,c5,dxc5,O-O,a3,Bxc5,Nf3,Qc7,Bg5,Be7,e4,d6,Be2,a6,O-O,Nbd7,Nd4,b6,Rac1,Bb7,Be3,Rfe8,f3,Ne5,Na4,Rab8,Qd2,Ng6,Kh1,Nd7,b4,Bf8,Rfd1,Rbd8,Nb3,Ba8,Qe1,Bc6,b5,Ba8,bxa6,Qa7,Nc3,Qxa6,Nb5,Rc8,c5,Qa4,Qb4,Qxb4,axb4,bxc5,Nxd6,Bxd6,Rxd6,c4,Rxc4,Rxc4,Bxc4,Nde5,Bf1,f5,Nc5,fxe4,fxe4,Nf8,h3,Nf7,Ra6,Ne5,b5,Ned7,Ra7,Nxc5,Bxc5,Bxe4,b6,Bc6,Rc7,Ba4,b7,Nd7,Rc8,Kf7,Bd6,h6,Be2,g6,Bd3,Rg8,Rxg8,Kxg8,Bxg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [38]:
# This function will be used to concatenate two strings from two arrays
def combineStrings(a1,a2):
  new_list = []
  for i in range(0, a1.size):
    new_list.append(a1[i] + a2[i])
  return new_list

In [39]:
# Creating a list of integer from 1 to 75
# Then covert list type from int to string
a = np.arange(1,76,1)
a_str = list(map(str,a))

In [40]:
# Create move piece names.
w_pieces = np.full(75, ['W'])
b_pieces = np.full(75, ['B'])
w_new_list = combineStrings(w_pieces, a_str)
b_new_list = combineStrings(b_pieces, a_str)

In [41]:
# Combine list but alternate between white and black piece.
values = [None] * (len(w_new_list) + len(b_new_list))
values[::2] = w_new_list
values[1::2] = b_new_list

In [42]:
# rename columns to show W1, B1, W2, B2, .....
keys = np.arange(0,150,1)
dictionary = dict(zip(keys,values))
chess_data.rename(columns = dictionary, inplace=True)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
7,2000,1-0,2850,2675,67,e4,c5,Nf3,d6,d4,Nf6,Nc3,cxd4,Nxd4,a6,Be3,Nbd7,f3,g6,Qd2,b5,a4,bxa4,Nxa4,Bg7,Be2,Bb7,c4,O-O,O-O,Nc5,b4,e5,bxc5,exd4,Bg5,Re8,cxd6,Qxd6,c5,Qc6,Bc4,Bc8,Nb6,Ra7,Rfc1,Nd7,Bd5,Qc7,Nc4,Qxc5,Na5,Qf8,Nc6,h6,Bh4,Rc7,Be7,Rxc6,Bxf8,Rxc1+,Rxc1,Nxf8,Qf4,Be6,Rc7,Bxd5,exd5,f5,d6,Rd8,Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,2001,1-0,2849,2629,93,c4,e5,Nc3,Nf6,Nf3,Nc6,g3,d5,cxd5,Nxd5,Bg2,Nb6,O-O,Be7,Rb1,O-O,b4,Nxb4,Nxe5,c6,a3,N4d5,Nxd5,cxd5,a4,Bf6,d4,Bf5,Rb5,a6,Rb2,Rc8,Bf4,Bxe5,Bxe5,Re8,e3,Re7,g4,Be4,f3,Nc4,Re2,Bg6,h4,f6,Bf4,Qd7,Rfe1,Rc6,h5,Bf7,e4,dxe4,Rxe4,Rxe4,Rxe4,Bd5,Re2,Rc8,h6,g5,Bg3,Re8,Qe1,Rxe2,Qxe2,Kf8,Qd3,Nb6,a5,Nc4,Bf1,Qe7,Qc3,Qe3+,Qxe3,Nxe3,Bd3,Nc4,Be1,Kg8,Kf2,b5,Bb4,Nb2,Bf5,Bc4,Ke3,Bd5,Bc8,Nc4+,Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19,2003,1-0,2847,2753,123,e4,c6,d4,d5,Nd2,dxe4,Nxe4,Bf5,Ng3,Bg6,h4,h6,Nf3,Nd7,h5,Bh7,Bd3,Bxd3,Qxd3,Ngf6,Bf4,e6,O-O-O,Be7,Kb1,O-O,Ne4,Nxe4,Qxe4,Nf6,Qe2,Qd5,Ne5,Qe4,Qxe4,Nxe4,Rhe1,Nf6,g4,Rfd8,Be3,Bd6,f3,Rac8,c4,a5,a4,Nd7,Bd2,Bc7,Bc3,Nxe5,dxe5,c5,Kc2,Rxd1,Kxd1,Rd8+,Ke2,Rd7,f4,Bd8,f5,Bg5,f6,b6,Ra1,Rd8,Ra3,gxf6,Rb3,Bf4,Rxb6,Bxe5,Bxe5,fxe5,Rb5,Rd4,Rxc5,Rxg4,b3,Rg3,Rxe5,Rxb3,Rxa5,f5,Ra8+,Kg7,c5,Rc3,Rc8,Ra3,c6,Rxa4,Re8,Rc4,Rxe6,f4,Rg6+,Kh7,Kd3,Rc5,Kd4,Rc1,Ke4,Rc4+,Kd5,Rc3,d4,c1,e4,c4+,f3,h8,xh6+,g7,d6,h7,g4,g7,d7+,f6,7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
21,2001,1-0,2838,2631,85,e4,c5,Nf3,e6,d4,cxd4,Nxd4,a6,Nc3,Qc7,Bd3,Nc6,Nxc6,bxc6,O-O,Nf6,Qe2,Bd6,f4,e5,Kh1,h6,Bd2,O-O,f5,Re8,Bc4,a5,Rf3,Bb4,Rg3,Kf8,Qe3,Ng8,f6,gxf6,Rf1,Bxc3,Rxg8+,Kxg8,Qg3+,Kf8,Rxf6,d5,Bxh6+,Ke7,Rxf7+,Kd6,Qg6+,Be6,Rxc7,Kxc7,exd5,cxd5,Bb5,Reb8,a4,Bd7,bxc3,Bxb5,axb5,Rxb5,h4,a4,Bg7,d4,Qf7+,Kb6,Qe6+,Kb7,Bxe5,Rxe5,Qxe5,a3,Qd5+,Kb8,cxd4,a2,Qb3+,Kc7,Qc3+,Kd7,Qa1,Kd6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
23,2001,1-0,2838,2802,99,d4,Nf6,c4,e6,Nc3,Bb4,Qc2,c5,dxc5,O-O,a3,Bxc5,Nf3,Qc7,Bg5,Be7,e4,d6,Be2,a6,O-O,Nbd7,Nd4,b6,Rac1,Bb7,Be3,Rfe8,f3,Ne5,Na4,Rab8,Qd2,Ng6,Kh1,Nd7,b4,Bf8,Rfd1,Rbd8,Nb3,Ba8,Qe1,Bc6,b5,Ba8,bxa6,Qa7,Nc3,Qxa6,Nb5,Rc8,c5,Qa4,Qb4,Qxb4,axb4,bxc5,Nxd6,Bxd6,Rxd6,c4,Rxc4,Rxc4,Bxc4,Nde5,Bf1,f5,Nc5,fxe4,fxe4,Nf8,h3,Nf7,Ra6,Ne5,b5,Ned7,Ra7,Nxc5,Bxc5,Bxe4,b6,Bc6,Rc7,Ba4,b7,Nd7,Rc8,Kf7,Bd6,h6,Be2,g6,Bd3,Rg8,Rxg8,Kxg8,Bxg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<

In [43]:
total_games = chess_data.index.values.size + 1

def createGames(total_games):
  new_list = []
  for i in range(1, total_games):
    new_list.append('Game' + str(i))
  return new_list

index_list = createGames(total_games)
chess_data.index = index_list

In [44]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
Game1,2000,1-0,2850,2675,67,e4,c5,Nf3,d6,d4,Nf6,Nc3,cxd4,Nxd4,a6,Be3,Nbd7,f3,g6,Qd2,b5,a4,bxa4,Nxa4,Bg7,Be2,Bb7,c4,O-O,O-O,Nc5,b4,e5,bxc5,exd4,Bg5,Re8,cxd6,Qxd6,c5,Qc6,Bc4,Bc8,Nb6,Ra7,Rfc1,Nd7,Bd5,Qc7,Nc4,Qxc5,Na5,Qf8,Nc6,h6,Bh4,Rc7,Be7,Rxc6,Bxf8,Rxc1+,Rxc1,Nxf8,Qf4,Be6,Rc7,Bxd5,exd5,f5,d6,Rd8,Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game2,2001,1-0,2849,2629,93,c4,e5,Nc3,Nf6,Nf3,Nc6,g3,d5,cxd5,Nxd5,Bg2,Nb6,O-O,Be7,Rb1,O-O,b4,Nxb4,Nxe5,c6,a3,N4d5,Nxd5,cxd5,a4,Bf6,d4,Bf5,Rb5,a6,Rb2,Rc8,Bf4,Bxe5,Bxe5,Re8,e3,Re7,g4,Be4,f3,Nc4,Re2,Bg6,h4,f6,Bf4,Qd7,Rfe1,Rc6,h5,Bf7,e4,dxe4,Rxe4,Rxe4,Rxe4,Bd5,Re2,Rc8,h6,g5,Bg3,Re8,Qe1,Rxe2,Qxe2,Kf8,Qd3,Nb6,a5,Nc4,Bf1,Qe7,Qc3,Qe3+,Qxe3,Nxe3,Bd3,Nc4,Be1,Kg8,Kf2,b5,Bb4,Nb2,Bf5,Bc4,Ke3,Bd5,Bc8,Nc4+,Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game3,2003,1-0,2847,2753,123,e4,c6,d4,d5,Nd2,dxe4,Nxe4,Bf5,Ng3,Bg6,h4,h6,Nf3,Nd7,h5,Bh7,Bd3,Bxd3,Qxd3,Ngf6,Bf4,e6,O-O-O,Be7,Kb1,O-O,Ne4,Nxe4,Qxe4,Nf6,Qe2,Qd5,Ne5,Qe4,Qxe4,Nxe4,Rhe1,Nf6,g4,Rfd8,Be3,Bd6,f3,Rac8,c4,a5,a4,Nd7,Bd2,Bc7,Bc3,Nxe5,dxe5,c5,Kc2,Rxd1,Kxd1,Rd8+,Ke2,Rd7,f4,Bd8,f5,Bg5,f6,b6,Ra1,Rd8,Ra3,gxf6,Rb3,Bf4,Rxb6,Bxe5,Bxe5,fxe5,Rb5,Rd4,Rxc5,Rxg4,b3,Rg3,Rxe5,Rxb3,Rxa5,f5,Ra8+,Kg7,c5,Rc3,Rc8,Ra3,c6,Rxa4,Re8,Rc4,Rxe6,f4,Rg6+,Kh7,Kd3,Rc5,Kd4,Rc1,Ke4,Rc4+,Kd5,Rc3,d4,c1,e4,c4+,f3,h8,xh6+,g7,d6,h7,g4,g7,d7+,f6,7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game4,2001,1-0,2838,2631,85,e4,c5,Nf3,e6,d4,cxd4,Nxd4,a6,Nc3,Qc7,Bd3,Nc6,Nxc6,bxc6,O-O,Nf6,Qe2,Bd6,f4,e5,Kh1,h6,Bd2,O-O,f5,Re8,Bc4,a5,Rf3,Bb4,Rg3,Kf8,Qe3,Ng8,f6,gxf6,Rf1,Bxc3,Rxg8+,Kxg8,Qg3+,Kf8,Rxf6,d5,Bxh6+,Ke7,Rxf7+,Kd6,Qg6+,Be6,Rxc7,Kxc7,exd5,cxd5,Bb5,Reb8,a4,Bd7,bxc3,Bxb5,axb5,Rxb5,h4,a4,Bg7,d4,Qf7+,Kb6,Qe6+,Kb7,Bxe5,Rxe5,Qxe5,a3,Qd5+,Kb8,cxd4,a2,Qb3+,Kc7,Qc3+,Kd7,Qa1,Kd6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game5,2001,1-0,2838,2802,99,d4,Nf6,c4,e6,Nc3,Bb4,Qc2,c5,dxc5,O-O,a3,Bxc5,Nf3,Qc7,Bg5,Be7,e4,d6,Be2,a6,O-O,Nbd7,Nd4,b6,Rac1,Bb7,Be3,Rfe8,f3,Ne5,Na4,Rab8,Qd2,Ng6,Kh1,Nd7,b4,Bf8,Rfd1,Rbd8,Nb3,Ba8,Qe1,Bc6,b5,Ba8,bxa6,Qa7,Nc3,Qxa6,Nb5,Rc8,c5,Qa4,Qb4,Qxb4,axb4,bxc5,Nxd6,Bxd6,Rxd6,c4,Rxc4,Rxc4,Bxc4,Nde5,Bf1,f5,Nc5,fxe4,fxe4,Nf8,h3,Nf7,Ra6,Ne5,b5,Ned7,Ra7,Nxc5,Bxc5,Bxe4,b6,Bc6,Rc7,Ba4,b7,Nd7,Rc8,Kf7,Bd6,h6,Be2,g6,Bd3,Rg8,Rxg8,Kxg8,Bxg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

We finally have a dataset that has been prepared and formatted to our needs.

In [45]:
chess_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 156644 entries, Game1 to Game156644
Columns: 155 entries, Date to B75
dtypes: int32(4), string(151)
memory usage: 184.0+ MB


In [46]:
chess_data.head(100)

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
Game1,2000,1-0,2850,2675,67,e4,c5,Nf3,d6,d4,Nf6,Nc3,cxd4,Nxd4,a6,Be3,Nbd7,f3,g6,Qd2,b5,a4,bxa4,Nxa4,Bg7,Be2,Bb7,c4,O-O,O-O,Nc5,b4,e5,bxc5,exd4,Bg5,Re8,cxd6,Qxd6,c5,Qc6,Bc4,Bc8,Nb6,Ra7,Rfc1,Nd7,Bd5,Qc7,Nc4,Qxc5,Na5,Qf8,Nc6,h6,Bh4,Rc7,Be7,Rxc6,Bxf8,Rxc1+,Rxc1,Nxf8,Qf4,Be6,Rc7,Bxd5,exd5,f5,d6,Rd8,Qh4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game2,2001,1-0,2849,2629,93,c4,e5,Nc3,Nf6,Nf3,Nc6,g3,d5,cxd5,Nxd5,Bg2,Nb6,O-O,Be7,Rb1,O-O,b4,Nxb4,Nxe5,c6,a3,N4d5,Nxd5,cxd5,a4,Bf6,d4,Bf5,Rb5,a6,Rb2,Rc8,Bf4,Bxe5,Bxe5,Re8,e3,Re7,g4,Be4,f3,Nc4,Re2,Bg6,h4,f6,Bf4,Qd7,Rfe1,Rc6,h5,Bf7,e4,dxe4,Rxe4,Rxe4,Rxe4,Bd5,Re2,Rc8,h6,g5,Bg3,Re8,Qe1,Rxe2,Qxe2,Kf8,Qd3,Nb6,a5,Nc4,Bf1,Qe7,Qc3,Qe3+,Qxe3,Nxe3,Bd3,Nc4,Be1,Kg8,Kf2,b5,Bb4,Nb2,Bf5,Bc4,Ke3,Bd5,Bc8,Nc4+,Ke2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game3,2003,1-0,2847,2753,123,e4,c6,d4,d5,Nd2,dxe4,Nxe4,Bf5,Ng3,Bg6,h4,h6,Nf3,Nd7,h5,Bh7,Bd3,Bxd3,Qxd3,Ngf6,Bf4,e6,O-O-O,Be7,Kb1,O-O,Ne4,Nxe4,Qxe4,Nf6,Qe2,Qd5,Ne5,Qe4,Qxe4,Nxe4,Rhe1,Nf6,g4,Rfd8,Be3,Bd6,f3,Rac8,c4,a5,a4,Nd7,Bd2,Bc7,Bc3,Nxe5,dxe5,c5,Kc2,Rxd1,Kxd1,Rd8+,Ke2,Rd7,f4,Bd8,f5,Bg5,f6,b6,Ra1,Rd8,Ra3,gxf6,Rb3,Bf4,Rxb6,Bxe5,Bxe5,fxe5,Rb5,Rd4,Rxc5,Rxg4,b3,Rg3,Rxe5,Rxb3,Rxa5,f5,Ra8+,Kg7,c5,Rc3,Rc8,Ra3,c6,Rxa4,Re8,Rc4,Rxe6,f4,Rg6+,Kh7,Kd3,Rc5,Kd4,Rc1,Ke4,Rc4+,Kd5,Rc3,d4,c1,e4,c4+,f3,h8,xh6+,g7,d6,h7,g4,g7,d7+,f6,7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game4,2001,1-0,2838,2631,85,e4,c5,Nf3,e6,d4,cxd4,Nxd4,a6,Nc3,Qc7,Bd3,Nc6,Nxc6,bxc6,O-O,Nf6,Qe2,Bd6,f4,e5,Kh1,h6,Bd2,O-O,f5,Re8,Bc4,a5,Rf3,Bb4,Rg3,Kf8,Qe3,Ng8,f6,gxf6,Rf1,Bxc3,Rxg8+,Kxg8,Qg3+,Kf8,Rxf6,d5,Bxh6+,Ke7,Rxf7+,Kd6,Qg6+,Be6,Rxc7,Kxc7,exd5,cxd5,Bb5,Reb8,a4,Bd7,bxc3,Bxb5,axb5,Rxb5,h4,a4,Bg7,d4,Qf7+,Kb6,Qe6+,Kb7,Bxe5,Rxe5,Qxe5,a3,Qd5+,Kb8,cxd4,a2,Qb3+,Kc7,Qc3+,Kd7,Qa1,Kd6,c4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Game5,2001,1-0,2838,2802,99,d4,Nf6,c4,e6,Nc3,Bb4,Qc2,c5,dxc5,O-O,a3,Bxc5,Nf3,Qc7,Bg5,Be7,e4,d6,Be2,a6,O-O,Nbd7,Nd4,b6,Rac1,Bb7,Be3,Rfe8,f3,Ne5,Na4,Rab8,Qd2,Ng6,Kh1,Nd7,b4,Bf8,Rfd1,Rbd8,Nb3,Ba8,Qe1,Bc6,b5,Ba8,bxa6,Qa7,Nc3,Qxa6,Nb5,Rc8,c5,Qa4,Qb4,Qxb4,axb4,bxc5,Nxd6,Bxd6,Rxd6,c4,Rxc4,Rxc4,Bxc4,Nde5,Bf1,f5,Nc5,fxe4,fxe4,Nf8,h3,Nf7,Ra6,Ne5,b5,Ned7,Ra7,Nxc5,Bxc5,Bxe4,b6,Bc6,Rc7,Ba4,b7,Nd7,Rc8,Kf7,Bd6,h6,Be2,g6,Bd3,Rg8,Rxg8,Kxg8,Bxg6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,

In [48]:
# conda install pandas = 1.4.2
print(pd.__version__)

1.4.2


In [45]:
# we will upload pkl file to github and use it for visualization
chess_data.to_pickle("./chess_data.pkl", compression = 'zip')

In [46]:
chess_data.to_csv('./chess_data.csv')